In [109]:
train_file="/home/harshit/Desktop/engtocmd/data/nkjcb_data.csv"
word_vec_file="/home/harshit/Desktop/engtocmd/data/wiki-news-300d-1M.vec"
txt_train_file="/home/harshit/Desktop/kjcb_data.txt"

In [125]:
with open(txt_train_file, "r", encoding="utf-8-sig") as f:        
    d=[]
    lines=f.readlines()    
    print(len(lines))
    for i,l in enumerate(lines):        
        l=l.strip()
        l=l.lower()
        if i%3!=0:
            d[i//3].append(l)
        else:
            d.append([i//3, l])            
print(d[0])
n=np.array(d)

177
[0, 'create a directory on the desktop', 'mkdir', 'o o o o o dirloc']


In [126]:
import csv
with open(train_file, "w+") as f:
    w=csv.writer(f, delimiter=",")
    w.writerow(["id","tokens","slots","intent"])
    for i,row in enumerate(d):
#         print(i)
        w.writerow([row[0], row[1], row[3], row[2]])
        if "directory" in row[1].split():
            w.writerow([row[0], row[1].replace("directory","folder"), row[3], row[2]])
        if "folder" in row[1].split():
            w.writerow([row[0], row[1].replace("folder","directory"), row[3], row[2]])

In [112]:
import pandas as pd
import numpy as np
import tensorflow
from tensorflow import keras
from tensorflow.keras.layers import *

In [127]:
df=pd.read_csv(train_file)
print(df.shape)
print(df.iloc[10])

(87, 4)
id                                    5
tokens    create a folder at dekstop/tp
slots                    o o o o dirloc
intent                            mkdir
Name: 10, dtype: object


In [128]:
df

,id,tokens,slots,intent
0,0,create a directory on the desktop,o o o o o dirloc,mkdir
1,0,create a folder on the desktop,o o o o o dirloc,mkdir
2,1,create a directory,o o o,mkdir
3,1,create a folder,o o o,mkdir
4,2,create a folder,o o o,mkdir
5,2,create a directory,o o o,mkdir
6,3,make a folder,o o o,mkdir
7,3,make a directory,o o o,mkdir
8,4,create a folder named abc at dekstop/tp,o o o o dirname o dirloc,mkdir
9,4,create a directory named abc at dekstop/tp,o o o o dirname o dirloc,mkdir


In [129]:
n=df.to_numpy()
intents=[]
slots=[]
for row in n:
    for i,val in enumerate(row):
        if i==3:
            if val not in intents:
                intents.append(val)
        elif i==2:
            for slot in val.split():
                if slot not in slots:
                    slots.append(slot)
print(len(intents))
print(intents)
print(len(slots))
print(slots)

7
['mkdir', 'cd', 'ls', 'touch', 'man', 'rmdir', 'cdback']
5
['o', 'dirloc', 'dirname', 'filname', 'comname']


In [104]:
import numpy as np
import io


def vocab(embf=None, n=100000, req=set()):
    if not embf:
        embf=word_vec_file
    def load_vectors(fname, n):
        fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
        # n, d = map(int, fin.readline().split())
        data = {}
        for i, line in enumerate(fin):
            if i == 0:
                continue            
            tokens = line.rstrip().split(' ')            
            if i<n:
                data[tokens[0]] = list(map(float, tokens[1:]))
            elif tokens[0] in req:
                data[tokens[0]] = list(map(float, tokens[1:]))
        return data

    word_to_vec = {}
    unk = [0] *300
    word_to_vec = load_vectors(embf, n)    
    return word_to_vec

In [105]:
def gen(train_set, intents, slots, word_to_vec):
    while True:        
        for row in train_set:            
            tokens=row[1].split()
            ip=np.zeros((1,len(tokens),300)) 
            for i, token in enumerate(tokens):
                if token in word_to_vec:
                    ip[0,i]=word_to_vec[token]
                else:
                    ip[0,i]=[0]*300
            op_intent=np.zeros((1, len(intents)))
            op_intent[0, intents.index(row[3])]=1
            op_slots=np.zeros((1,len(tokens),len(slots)))
            r_slots=row[2].split()
            if not len(r_slots) == len(tokens):
              print(tokens)
              print(r_slots)
              continue
            for i,slot in enumerate(r_slots):
                op_slots[0,i,slots.index(slot)]=1
                
            yield ip, (op_intent, op_slots)

In [7]:
word_to_vec=vocab(n=1*10**5)
g=gen(n, intents, slots, word_to_vec)
ip, op=g.__next__()
print(op[0])
print(op[1])

[[1. 0. 0. 0. 0.]]
[[[1. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0.]
  [0. 1. 0. 0. 0.]]]


In [87]:
len(word_to_vec)

99999

In [130]:
n=df.to_numpy()
ip=Input(name="Input",batch_shape=(1,None,300))
enc_seq, enc_final_state=GRU(units=100, name="Encoder", return_sequences=True, return_state=True)(ip)
intent_classifier=Dense(units=len(intents), activation="softmax", name="intent_classifier")(enc_final_state)

decoder_seq=GRU(units=100 ,return_sequences=True, name="Decoder")(enc_seq, initial_state=enc_final_state)
slots_tagger=Dense(units=len(slots), activation="softmax", name="slots_tagger")(decoder_seq)
model=keras.models.Model(inputs=ip,outputs=(intent_classifier, slots_tagger),name="intent_and_slots")
model.summary()
# input()

# word_to_vec=vocab(n=1*10**5)
model.compile(optimizer=keras.optimizers.Adam(), loss="categorical_crossentropy", loss_weights=[1.0, 1.0])
model.fit_generator(gen(n, intents, slots, word_to_vec), epochs=13, steps_per_epoch=n.shape[0])

Model: "intent_and_slots"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input (InputLayer)              [(1, None, 300)]     0                                            
__________________________________________________________________________________________________
Encoder (GRU)                   [(1, None, 100), (No 120300      Input[0][0]                      
__________________________________________________________________________________________________
Decoder (GRU)                   (1, None, 100)       60300       Encoder[0][0]                    
                                                                 Encoder[0][1]                    
__________________________________________________________________________________________________
intent_classifier (Dense)       (None, 7)            707         Encoder[0][1]     

In [131]:
while True:
    sent=input("Enter\n")
    if sent == "-1":
        break
    tokens=sent.split()
    ip=np.zeros((1,len(tokens),300)) 
    for i, token in enumerate(tokens):
        if token in word_to_vec:
            ip[0,i]=word_to_vec[token]
        else:
            ip[0,i]=[0]*300
    pred=model.predict(ip)
    intents_pred=pred[0]
    intents_pred=list(np.squeeze(intents_pred))
    print(intents_pred)
    print("Intent:", intents[intents_pred.index(max(intents_pred))])
    slots_pred=pred[1]
    slots_pred=np.squeeze(slots_pred)
    for pred in slots_pred:
        pred=list(pred)        
        print(slots[pred.index(max(pred))], end=" ")

Enter
move to C
[0.00081383437, 0.9809436, 3.355895e-05, 0.00054370065, 0.00097459205, 0.00057141477, 0.01611941]
Intent: cd
o o dirname Enter
move to temp
[0.00067989586, 0.97915936, 2.2113967e-05, 0.00039332715, 0.00044277395, 0.0006223811, 0.018680193]
Intent: cd
o o dirname Enter
move to Desktop
[0.00073203, 0.9951351, 2.097654e-05, 0.00019925727, 8.992551e-05, 0.0007060647, 0.003116498]
Intent: cd
o o dirname Enter
move back
[0.0008465664, 0.04641577, 0.000107593994, 0.0030690925, 0.0019730506, 0.0021744443, 0.9454134]
Intent: cdback
o o Enter
go back
[0.00081240357, 0.019262802, 0.0005725477, 0.003432948, 0.004694367, 0.0017037158, 0.9695212]
Intent: cdback
o o Enter
create a file
[0.08959469, 0.0021672472, 0.0010264333, 0.90512353, 0.00048715863, 0.00026609475, 0.001334779]
Intent: touch
o o o Enter
make a file
[0.06881896, 0.0034016524, 0.0038997051, 0.9192958, 0.0015348527, 0.0003061135, 0.0027429361]
Intent: touch
o o o Enter
-1


In [132]:
model.save("/home/harshit/Desktop/engtocmd/data/kj_cb-e13-nd100k.h5")